In [ ]:
from nilearn import plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import matplotlib as mpl
#mpl.rcParams['figure.dpi']= 200  

# Setup brainscapes

### Set a EBRAINS Knowledge Graph access token

Brainscapes retrieves some data from the EBRAINS Knowledge Graph, which requires
authentication. To do so, please follow these steps:

 1. If you do not yet have an EBRAINS account, register [here](https://ebrains.eu/register). As you are reading this notebook in the EBRAINS collaboratory, this is most probably not necessary at this point.
 2. Your EBRAINS account needs to be enabled for programmatic access to the EBRAINS Knowledge Graph to fetch metadata. This is formal step to acknowledge additional terms of use, and done quickly by emailing to the KG team. A link and template email to do so can be found right on top of the [Knowledge Graph developer page](https://kg.humanbrainproject.eu/develop.html).
 3. Create an authentication token for EBRAINS by visiting
[the EBRAINS authorization endpoint](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize). 
 4. Copy the token, and store it in the enviroment variable `HBP_AUTH_TOKEN` (just modify and execute the cell below accordingly).

Note that as of now, you have to get a new token (steps 3. and 4.) approximately every day.

In [ ]:
import webbrowser
from os import environ
webbrowser.open('https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize')
token = input("Enter your token here: ")
environ['HBP_AUTH_TOKEN'] = token

### Optional: adjust local data cache

Brainscapes maintains a local cache of retrieved data. It will automatically choose the system default of your user account on most common operating systems, but you an also choose an explicit folder by setting the environment variable `BRAINSCAPES_CACHEDIR`. We are not using this here.

In [ ]:
#!mkdir -p /tmp/brainscapescache
#environ['BRAINSCAPES_CACHEDIR'] = "/tmp/brainscapescache"

### Load the library

In [ ]:
import brainscapes as bs
bs.logger.setLevel("INFO") # show us some messages
#bs.clear_cache()

# Choosing a parcellation

We first load and show the previous Julich-Brain version, then the most recent one.

In [ ]:
atlas = bs.atlases.MULTILEVEL_HUMAN_ATLAS
atlas.select_parcellation(bs.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5)

# in MNI 152 space
icbm_mri = atlas.get_template(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
icbm_maps = atlas.get_maps(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
plotting.plot_stat_map(icbm_maps['left hemisphere'])

# bigbrain
bigbrain_template = atlas.get_template(bs.spaces.BIG_BRAIN_HISTOLOGY,resolution=640)
bigbrain_map = atlas.get_maps(bs.spaces.BIG_BRAIN_HISTOLOGY,resolution=640)
plotting.plot_stat_map(bigbrain_map,bg_img=bigbrain_template)

# Selecting brain regions from an atlas

To select atlas regions, an explicit region identifier or unique name and key can be used. If an arbitrary string is passed, the client  does its best to identify the corresponding region. It will complain if that is not possible.

In [ ]:
# we can just give a string and see if the system can disambiguiate it
atlas.select_region("v1")
print("Selected region from 'v1' is",atlas.selected_region)

print('v1 includes the left and right hemisphere!')
print(repr(atlas.selected_region))

# we can be more specific easily
atlas.select_region("v1 left")
print("Selected region from 'v1 left' is",atlas.selected_region)

# we can also auto-complete on the 'regionnames' attribute of the atlas 
# - this immediately leads to a unique selection
atlas.select_region(atlas.selected_parcellation.regionnames.AREA_HOC1_V1_17_CALCS_LEFT_HEMISPHERE)

We can also search the region hierarchy to explore available regions:

In [ ]:
matching_regions = atlas.selected_parcellation.regions.find('ifg')
print("Regions found for search string 'ifg':")
print("\n".join(" - "+str(r) for r in matching_regions))

Brainscapes supports access to continuous maps for some parcellations. For the Julich-Brain, this translates to the actual probability maps of each area. Let's look at a probability map of the frontal pole in the ICBM space.

In [ ]:
atlas.select_region('fp2 right')
pmap = atlas.selected_region.get_specific_map(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
plotting.plot_roi(pmap)

# Extracting brain region features from EBRAINS

We can run queries from any selected subtree in the region hierarchy, but here we select a particular cortical region from the frontal lobe .

### Extract spatial properties of brain regions

In [ ]:
props = atlas.regionprops(
    bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
for prop in props:
    print(prop)

### Extract transmitter receptor densities

Transmitter receptor density fingerprints are linked to brain regions by their name in the EBRAINS Knowledge Graph. Like any data feature, they are accessed using the `query_data` method of the atlas, which makes use of the current selection in the atlas. The `query_data` method knows from the specified data modality that the match is determined from the brain region identified. Receptor densities come as a nicely structured datatype. Amongst other things, they can visualize themselves in a plot.

In [ ]:
atlas.select_region('v1 left')
features = atlas.query_data(
    bs.features.modalities.ReceptorDistribution)
for r in features:
    fig = r.plot(r.region)

### Extracting Gene Expressions from the Allen Atlas 

The atlas client can make calls to gene expression data from the Allen atlas and evaluate them in the ICBM space to find regional gene expression levels. It also has a list of available gene names for convenient selection. Gene expressions are linked to atlas regions by coordinates of their probes in MNI space. The `query_data` method detects this from the feature modality, and applies the mask of the regions that are currently selected in the atlas to filter the probes. We can visualize these filtered locations.

In [ ]:
features = atlas.query_data(
    bs.features.modalities.GeneExpression, 
    gene=bs.features.gene_names.GABARAPL2)
print(features[0])

# plot
all_coords = [tuple(g.location) for g in features]
mask = atlas.get_mask(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
display = plotting.plot_roi(mask)
display.add_markers(all_coords,marker_size=5) 

### Retrieving a connectivity matrix
For brainscapes, a connectivity matrix is a data feature like the others below, and can be found using the same `query_data` function just by choosing another modality. Brainscapes knows from the modality type that this type of data does not match to the selected brain region in the atlas, but to the selected parcellation. Therefore, `query_data` return connectivity datasets that are defined for the selected parcellation. 

To learn about the nature of the provided connectivity, the `src_info` attribute provides a detailed description of the dataset. 

In [ ]:
# Get the first four connectivity matrices available for the parcellation
features = atlas.query_data(bs.features.modalities.ConnectivityMatrix)[:4]

# format dataset names for use as figure titles
from textwrap import wrap
titleformat = lambda text : "\n".join(wrap(text.replace('_',' '),20)) 

# plot the matrices
f,axs = plt.subplots(1,len(features))
for i,feature in enumerate(features):
    axs[i].imshow(feature.matrix,cmap=plt.cm.viridis)
    axs[i].set_title(titleformat(feature.src_name),size=8)
f.show()